In [1]:
library(viper)

Loading required package: Biobase

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.




Merge the regulons from each class of TF and then run either RegProcess or aracne2regulon, whichever one works

In [ ]:
RegProcess('tutorial/pbmc_r1-net-final.tsv', cpm.mat, out.dir = 'tutorial/', out.name = 'pbmc_r1-net-')

In [ ]:
#MDA_non_bulk <- readRDS(file = "../results/spher_nets/aracne/all_cells-meta.rds")
sct.mat <- MDA_non_bulk@assays$SCT@scale.data

In [ ]:
regulon <- aracne2regulon("/cellar/users/aklie/projects/collab/stephanie/2020_08_05_spheroid_vs_network/results/spher_nets/aracne/unsupervised_out/unsupervised_grn.txt", 
                          sct.mat)

In [ ]:
MDA.vip <- viper(sct.mat, regulon, method = 'none')

## UMAP vis of this matrix

In [ ]:
library(umap)
custom.config = umap.defaults
custom.config$random_state = 1

In [ ]:
vip.umap <- CustomUMAP(MDA.vip)

In [ ]:
## scatter plot
plot.dat <- data.frame('UMAP1' = vip.umap$layout[names(opt.clust),1], 
                       'UMAP2' = vip.umap$layout[names(opt.clust),2],
                       'Cluster' = as.factor(opt.clust),
                       'Phenotype' = MDA_non_bulk@meta.data["true.ident"])
ggplot(plot.dat, aes(UMAP1, UMAP2)) + geom_point(aes(color = true.ident)) + ggtitle('MDA Clusters')
ggplot(plot.dat, aes(UMAP1, UMAP2)) + geom_point(aes(color = Cluster)) + ggtitle('MDA Clusters')

## Heatmap visualization

In [ ]:
phenos <- factor(MDA_non_bulk@meta.data$true.ident)
vip.mrs <- MWUMrs(MDA.vip, clust.vect = phenos)

In [ ]:
## create plot matrix
num.mrs <- 25
cell.order <- names(phenos)
mr.set <- sapply(vip.mrs, function(x) {names(x$positive[1:num.mrs])})
mr.set <- unique(unlist(as.list(mr.set)))
#plot.mat <- MDA.vip[,cell.order]
plot.mat <- MDA.vip[mr.set, cell.order]

In [ ]:
# set colors and annotations
mat.breaks <- QuantileBreaks(plot.mat)
annot.df <- data.frame('Phenotype' = phenos)
#clust.colors <- ClusterColors(length(unique(opt.clust))); names(clust.colors) <- unique(opt.clust)
pheno.colors <- c("#F8766D", "#00BFC4")
names(pheno.colors) <- c("Network", "Spheroid")
annot.color <- list('Phenotype' = pheno.colors)

In [ ]:
## generate pheatmap
pheatmap(plot.mat, main = 'Viper Clustering: Master Regulators', fontsize = 20,
         annotation_col = annot.df, annotation_colors = annot.color,
         cluster_cols = FALSE, show_colnames = FALSE,
         cluster_rows = FALSE, show_rownames = TRUE, fontsize_row = 8,
         breaks = mat.breaks, color = ColorLevels(length(mat.breaks) - 1, 'vip'))